In [5]:
# imports
import json
import numpy as np
import os
import signal
import subprocess
import time

from nltk.tokenize import sent_tokenize
from stanza.server import CoreNLPClient

In [2]:
def clear_port_9000():
    # clear the 9000 port to be used by the CoreNLPClient
    command = "netstat -ano | findstr 9000"
    c = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr = subprocess.PIPE)
    stdout, stderr = c.communicate()
    if len(stdout) > 0:    
        # process is running on port 9000, terminate it
        print("Netstat response:")
        print(stdout)
        print("-----------------------------------------------------")
        pid = input("Enter the process ID of the process that should be terminated, or an alphanumeric character in case it is unnecessary.")
        if not pid.isalpha():
            os.kill(int(pid), signal.SIGTERM)
            time.sleep(5)  # wait for the process to close

In [3]:
with open('database_dump_drugs/0.json') as file:
    article = json.load(file)[0]['article_content'][:423]

In [13]:
sentences = [sent for sent in sent_tokenize(article)]

clear_port_9000()
with CoreNLPClient(properties='corenlp_server-2e15724b8064491b.props', memory='8G', be_quiet=True) as client:
    matches = client.tregex(text=sentences[0], pattern='NP')  # finds the noun phrases in the given text

# extract the noun phrases and their indices
noun_phrases = [(text, begin, end) for text, begin, end in
            zip([sentence[match_id]['spanString'] for sentence in matches['sentences'] for match_id in sentence],
                [sentence[match_id]['characterOffsetBegin'] for sentence in matches['sentences'] for match_id in sentence],
                [sentence[match_id]['characterOffsetEnd'] for sentence in matches['sentences'] for match_id in sentence])]

Netstat response:
b'  TCP    127.0.0.1:62869        127.0.0.1:9000         TIME_WAIT       0\r\n'
-----------------------------------------------------


2021-07-19 09:43:17 INFO: Starting server with command: java -Xmx8G -cp C:\Users\timjo\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-2e15724b8064491b.props -preload -outputFormat serialized


In [17]:
noun_phrases

[('An Ethiopian foreign national', 0, 29),
 ('eMzinoni', 46, 54),
 ('drugs at his tuck shop', 70, 92),
 ('drugs', 70, 75),
 ('his tuck shop', 79, 92),
 ('Friday, 6', 96, 105)]

In [16]:
# find head words that contain other head words
multiples = []
for text1, i1, i2 in noun_phrases:
    for text2, j1, j2 in noun_phrases:
        if (text1 != text2) and (i1 != j1) and (i2 != j2):
            if ((i1 >= j1) & (i2 < j2)) or ((i1 > j1) & (i2 <= j2)):
                multiples.append(i2)
set(multiples)

# remove these 'multiples'
# head_words = {key: val for key, val in noun_phrases.items() if val not in multiples}

set()